In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import colorsys
from sklearn.linear_model import LinearRegression

# 周波数
sr = 44100
# FFTのサンプル数
n_fft = 2048 

# オーディオファイルをモノラル信号として読み込む
y, sr = librosa.load(r'ここに曲のパス', sr=sr, mono=True ,dtype=np.float64)
# 特徴量を１つ算出するのに使うサンプル数
frame_length = 2205
# 何サンプルずらして特徴量を算出するかを決める変数
hop_length   =  1024
# スペクトログラムを計算する
S = np.abs(librosa.stft(y))

In [ ]:
# RMSを計算する
rms  = librosa.feature.rms(y=y,S=S,frame_length=frame_length, hop_length=hop_length)[0]
# RMSの時間軸を取得する
rms_time = np.floor(librosa.times_like(rms, sr=sr, hop_length=hop_length))

In [ ]:
# スペクトラルロールオフを計算する割合（通常は0.85や0.95が使われる）
roll_percent = 0.85

# スペクトラルロールオフ : 総スペクトラルエネルギーの低い周波数の割合（全体の85％が含まれる周波数）を算出
sro = librosa.feature.spectral_rolloff(y=y,S=S, sr=sr, hop_length=hop_length)[0]
# スペクトラルロールオフの時間軸を取得する
sroTimes = np.floor(librosa.times_like(sro, hop_length=hop_length, sr=sr))

In [ ]:
# メル周波数スペクトログラムを計算する
mfccs = librosa.feature.mfcc(y=y,S=S, sr=sr)
# メル周波数スペクトログラムの最大値を取得する
mfccs_max = np.max(mfccs, axis=0)

In [ ]:
# スペクトル重心を計算する
sc   = librosa.feature.spectral_centroid(y=y,S=S, n_fft=frame_length, hop_length=hop_length)[0]
# スペクトル重心の特定フレームでの時間を取得
scTimes = np.floor(librosa.times_like(sc, hop_length=hop_length, sr=sr))

In [ ]:
# ゼロ交差数(音の波の変化点)を計算する
zc = librosa.zero_crossings(y, pad=False)
zc_times = np.floor(librosa.frames_to_time(zc, hop_length=hop_length))

In [ ]:
# 時間の計測（現状合わない）
print(len(scTimes))
print(scTimes[-1])
print(len(rms))
print(rms_time[-1])
print(len(zc_times))
print(zc_times[-1])

In [ ]:
# 数値を正規化する
def normalize(data,min,max):
    normalized_data = []
    for i in data:
        if i >= max:
            i = max
        elif i <= min:
            i = min
        i = i/max
        normalized_data.append(i)
    return normalized_data

# スペクトルロールオフを正規化する
def sro_normalize(data):
    sro_max = 13000
    sro_min = 900
    normalized_data = normalize(data,sro_min,sro_max)
    return normalized_data

# スペクトル重心を正規化する
def sc_normalize(data):
    sc_max = 3000
    sc_min = 300
    normalized_data = normalize(data,sc_min,sc_max)
    return normalized_data

# メル周波数スペクトログラムを正規化する
def mfccs_color(mfccs_max):
    max = np.max(mfccs_max)
    mfccs_color = normalize(mfccs_max,0,max)
    return mfccs_color

In [ ]:
# 正規化したデータを使ってHSVに変換する
def color_HSV(sc,sro):
    color = np.zeros((len(sc),3))
    hue = sc_normalize(sc)
    value = sro_normalize(sro)
    saturation = mfccs_color(mfccs_max)
    for i in range(0,len(sc)):
        color[i] = [value[i],saturation[i],hue[i]]
    hue_plotter(saturation,hue,value)
    return color
# デバッグのためにグラフを描画する
def hue_plotter(saturation,hue,value):
    plt.plot(saturation,label="saturation")
    plt.plot(value,label="value")
    plt.plot(hue,label="hue")
    plt.legend()

# HSVをRGBに変換する
def color_HSV2color_RGB(color_HSV):
    color_RGB = [colorsys.hsv_to_rgb(h, s, b) for h, s, b in color_HSV]
    return color_RGB
color_HSV = color_HSV(sc,sro)
color_RGB = color_HSV2color_RGB(color_HSV)
#使えそうな1行 sc =[t / np.max(zsc for t in [sc]


In [ ]:
# 新しいグラフを描画
plt.figure()           
plt.bar(sroTimes,sro,color = color_RGB)

# 新しいグラフを描画
plt.figure()                                                                                
plt.bar(scTimes,sc,color = color_RGB)

In [ ]:
二元的な描画
plt.scatter(sc, sro, c=color_RGB, alpha=0.5)